In [1]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import (ConnectDrakeVisualizer, SceneGraph)
from pydrake.lcm import DrakeLcm
from pydrake.multibody.rigid_body_tree import (RigidBodyTree, AddModelInstancesFromSdfFile,
                                               FloatingBaseType,AddModelInstanceFromUrdfFile,AddFlatTerrainToWorld, RigidBodyFrame,AddModelInstanceFromUrdfStringSearchingInRosPackages)
from pydrake.multibody.multibody_tree import UniformGravityFieldElement, MultibodyTree,BodyIndex
from pydrake.multibody.multibody_tree.multibody_plant import MultibodyPlant
from pydrake.multibody.multibody_tree.parsing import AddModelFromSdfFile 
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator
from pydrake.common import (set_assertion_failure_to_throw_exception)
from pydrake.systems.controllers import (RbtInverseDynamicsController)
from pydrake.systems.primitives import (ConstantVectorSource, SignalLogger, TrajectorySource)
from pydrake.systems.meshcat_visualizer import MeshcatVisualizer
from pydrake.multibody.parsers import PackageMap
from pydrake.systems.framework import LeafSystem, PortDataType,BasicVector

from pydrake.multibody.rigid_body_plant import DrakeVisualizer,RigidBodyPlant
import numpy as np
import matplotlib.pyplot as plt
 
import os
import pydrake

from pydrake.systems.all import LcmSubscriberSystem, LcmPublisherSystem,AbstractValue
from pydrake.multibody.rigid_body_plant import ContactResults 
from pydrake.all import PySerializer
#from pydrake.all import  KinematicsResults

In [2]:
from lcmt import *
model_path = os.path.join(os.getcwd(), 'Model/LittleDog.urdf')

dt = 1e-3

lcm = DrakeLcm()
builder = DiagramBuilder()



In [3]:
class RobotCommandToRigidBodyPlantConverter(LeafSystem):
    def __init__(self, actuators):
        LeafSystem.__init__(self)
        self.num_actuators = len(actuators)
        
        self.robot_command_port_index = self._DeclareAbstractInputPort('robot_command_port', 
                                                                       AbstractValue.Make(littledog_command_t) ).get_index()
        self.desired_effort_port_indices = self._DeclareVectorOutputPort(BasicVector(self.num_actuators),
                                                                        self.OutputActuation).get_index()
 
    def OutputActuation(self, context, output):
        ## OutputDesiredEffort is not equal to output command
        msg = self. EvalAbstractInput(context, self.robot_command_port_index).get_value()      
        #msg = self.EvalVectorInput(context, self.robot_command_port_index).get_value()
        
        # TODO : Considering motor model (motor effort length etc.)
        output_command = msg.joint_command
        
        output.SetFromVector(np.array(output_command))

In [4]:
class RobotStateEncoder(LeafSystem):
    """
    RigidBodyPlant.KinematicsResults is not found
    """
    def __init__(self, rb_tree):
        LeafSystem.__init__(self)
        self.rb_tree = rb_tree
        self.num_controlled_q_ = self.rb_tree.get_num_positions()

        # Input Port
        self.joint_state_results_port_index = self._DeclareInputPort( 'joint_state_results_port', PortDataType.kVectorValued,
                                                        self.num_controlled_q_ *2).get_index()
    #         self.contact_results_port_index = self._DeclareAbstractInputPort('contact_results_port', 
    #                                                                        AbstractValue.Make(ContactResults)).get_index()
        # Output Port
        self.lcm_message_port_index = self._DeclareAbstractOutputPort('state_output_port',
                                                                      self.Allocator,
                                                                      self.OutputRobotState,
                                                                     ).get_index()  
    def Allocator(self):
        return AbstractValue.Make(robot_state_t)
    def OutputRobotState(self, context, output):
        #message = robot_state_t
#         message.timestamp = context.get_time()*1e6

#         #contact_result = EvalAbstractInput(context, contact_results_port_index).get_value()
#         joint_state_result = self.EvalVectorInput(context, self.joint_state_results_port_index).get_value()

#         message.num_joints = self.num_controlled_q_
#         message.joint_positon = joint_state_result[:self.num_controlled_q_]
#         message.joint_velocity = joint_state_result[self.num_controlled_q_:]
#         print(message.timestamp, message.num_joints, message.joint_positon, message.joint_velocity)
#         output.set_value(message)

        message = output.get_mutable_value()
        message.timestamp = context.get_time()*1e6

        #contact_result = EvalAbstractInput(context, contact_results_port_index).get_value()
        joint_state_result = self.EvalVectorInput(context, self.joint_state_results_port_index).get_value()

        message.num_joints = self.num_controlled_q_
        message.joint_positon = joint_state_result[:self.num_controlled_q_]
        message.joint_velocity = joint_state_result[self.num_controlled_q_:]
        print(message.timestamp, message.num_joints, message.joint_positon, message.joint_velocity)
        output.set_value(message)
         
#         output.timestamp = context.get_time()*1e6

#         #contact_result = EvalAbstractInput(context, contact_results_port_index).get_value()
#         joint_state_result = self.EvalVectorInput(context, self.joint_state_results_port_index).get_value()

#         output.num_joints = self.num_controlled_q_
#         output.joint_positon = joint_state_result[:self.num_controlled_q_]
#         output.joint_velocity = joint_state_result[self.num_controlled_q_:]
#         print(output.timestamp, output.num_joints, output.joint_positon, output.joint_velocity)
        #output.set_value(message)
    def joint_state_results_port(self):
        return self.get_input_port(self.joint_state_results_port_index)
    
        
         

In [5]:
# Build up your Robot World
rb_tree = RigidBodyTree()
world_frame = RigidBodyFrame("world_frame", rb_tree.world(), [0, 0, 0], [0, 0, 0])
AddFlatTerrainToWorld(rb_tree, 1000, 10)
robot_frame = RigidBodyFrame("robot_frame", rb_tree.world(), [0, 0, 0.5], [0, 0, 0])

# insert a robot from urdf files
pmap = PackageMap()
pmap.PopulateFromFolder(os.path.dirname(model_path))
AddModelInstanceFromUrdfStringSearchingInRosPackages(
        open(model_path, 'r').read(),
        pmap,
        os.path.dirname(model_path),
        FloatingBaseType.kRollPitchYaw,
        robot_frame,
        rb_tree)

{u'LittleDog': 0}

In [6]:
rb_tree.get_num_positions()

18

In [7]:
# RigidBodyPlant 
plant = builder.AddSystem(RigidBodyPlant(rb_tree, dt))
plant.set_name('plant')

# Robot command subscriber

#robot_command_subscriber = builder.AddSystem(LcmSubscriberSystem('ROBOT_COMMAND', PySerializer(littledog_command_t), lcm))
robot_command_subscriber = builder.AddSystem(LcmSubscriberSystem.Make('ROBOT_COMMAND', littledog_command_t, lcm))
robot_command_subscriber.set_name('robot_command_subscriber')

# Robot command to Plant Converter
robot_command_to_rigidbodyplant_converter = builder.AddSystem(RobotCommandToRigidBodyPlantConverter(rb_tree.actuators)) # input argu: rigidbody actuators
robot_command_to_rigidbodyplant_converter.set_name('robot_command_to_rigidbodyplant_converter')

# Visualizer
visualizer_publisher = builder.AddSystem(DrakeVisualizer(rb_tree, lcm ))
visualizer_publisher.set_name('visualizer_publisher')
visualizer_publisher.set_publish_period(1e-3);

# Robot State Encoder
robot_state_encoder = builder.AddSystem(RobotStateEncoder(rb_tree)) #force_sensor_info
robot_state_encoder.set_name('robot_state_encoder')

# Robot State Publisher
#robot_state_publisher = builder.AddSystem(LcmPublisherSystem('EST_ROBOT_STATE', PySerializer(robot_state_t), lcm))
robot_state_publisher = builder.AddSystem(LcmPublisherSystem.Make('EST_ROBOT_STATE', robot_state_t, lcm))
robot_state_publisher.set_name('robot_state_publisher')
robot_state_publisher.set_publish_period(1e-3);

# Connect everything
 
builder.Connect(robot_command_subscriber.get_output_port(0),
                robot_command_to_rigidbodyplant_converter.get_input_port(0))
builder.Connect(robot_command_to_rigidbodyplant_converter.get_output_port(0),
                plant.get_input_port(0))
builder.Connect(plant.state_output_port(),
                visualizer_publisher.get_input_port(0))
builder.Connect(plant.state_output_port(), #kinematics_results_output_port(0),
                robot_state_encoder.joint_state_results_port())
builder.Connect(robot_state_encoder.get_output_port(0),
                robot_state_publisher.get_input_port(0))


In [8]:
robot_state_t.num_joints

<member 'num_joints' of 'robot_state_t' objects>

In [9]:
diagram = builder.Build()


position = np.array(msg.position)
        velocity = np.array(msg.velocity)
        effort = np.array(msg.effort)
        k_q_p = np.array(msg.k_q_p)
        k_q_i = np.array(msg.k_q_i)
        k_q_d = np.array(msg.k_q_d)
        k_qd_d = np.array(msg.k_qd_d)
        k_f_p = np.array(msg.k_f_p)
        ff_qd = np.array(msg.ff_qd)
        ff_qd_d = np.array(msg.ff_qd_d)
        ff_const = np.array(msg.ff_const)
        
        output_command = k_q_p *(q_d -q) + k_q_i * (q_d - q) +  
                         k_qd_p *(qd_d -qd) + k_f_p * (f_d - f) 
                         + ff_qd * qd + ff_f_d * fd +ff_const

In [10]:
diagram_context = diagram.CreateDefaultContext()
simulator = Simulator(diagram, diagram_context)
simulator.set_publish_every_time_step(False)
simulation_time = 10


In [11]:

# simulator.set_target_realtime_rate(1)
# #simulator.Initialize()
# simulator.get_mutable_integrator().set_target_accuracy(1e-3)

# 

In [12]:
lcm.StartReceiveThread()
simulator.StepTo(simulation_time)
lcm.StopReceiveThread()

(0.0, 18, array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.]))


AssertionError: 